**Date**: June 28, 2025

**Author**: Zoey Liu

**Purpose** A script for automatic checking of some, but not all, annotation errors for Storiza project

In [93]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!ls

'All Computational Models are Wrong'
 childes_parsing.ipynb
'Coding & Recitation - Oct 26.ipynb'
'Copy of CommonVoice TR Edited Wav2Vec2 Workthough'
 crosslinguistic_nli.ipynb
 Expelliarmus.ipynb
 export_157513_project-157513-at-2025-07-01-07-44-30c2d71c.csv
 heatmaps.ipynb
 nli_classification.ipynb
 nli_dep_features.ipynb
 nli.ipynb
 OpenNMT-py.ipynb
 OpenNMT-tf.ipynb
 parseCHILDES.ipynb
 regress.ipynb
 Storiza_annotation_check.ipynb
 uniparse.ipynb
 Untitled0.ipynb
 uuparser.ipynb


In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/'
/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
import json, ast
import statistics
from difflib import SequenceMatcher

**Word-level csv file checks**

In [ ]:
word_level_data = pd.read_csv('export_157618_project-157618-at-2025-07-17-08-51-2c7ba368.csv', delimiter = ',')
word_level_data.columns

Index(['DisfluencyErrorType', 'GrammaticalErrorType', 'IPAHints',
       'MixedErrorTaxonomy', 'OrthographicErrorType', 'PhonologicalErrorType',
       'StructuralErrorType', 'VisualTrackingErrorType', 'WordAnnotation',
       '__id__', 'actual', 'agreement', 'annotation_id', 'annotator',
       'annotator_id', 'audio', 'content', 'created_at', 'end_time',
       'goldStandard', 'grade', 'id', 'issues', 'lead_time',
       'mispronunciation_word', 'nonchild', 'original_audio_name', 'picture',
       'produced_word', 'repeated', 'runon', 'segment_time',
       'sentence_level_id', 'sound', 'spoken_word', 'spoken_words',
       'start_time', 'time', 'title', 'topic', 'updated_at', 'userId',
       'words'],
      dtype='object')

In [ ]:
id_list = word_level_data['id'].tolist()
sentence_id_list = word_level_data['sentence_level_id'].tolist()
goldStandard_list = word_level_data['goldStandard'].tolist()

In [ ]:
repeated_sentences = [
    "One day, Whiskers found a hat on the mat",
    "The hat was big and flat",
    "Roy was sad but then spotted it on the shore",
    "In a test of speed, he raced against the tallest kids",
    "They cheered as he got faster at each turn",
    "He spies a bright site by the pine",
    "Look, a map",
    "He pulled out a map.",
    "Tim wanted to check",
    "Rusty loved to uncover hidden spots under the big, shady trees",
    "Her art and work are both neat",
    "A gentle breeze brushed his cheek",
    "After leaning back on the grass, he closed his eyes",
    "They loved exploring the endless woods, finding colorful flowers and watching birds",
    "His best friend was a small, mindful rabbit called Rue",
    "The race began, and initially, Pizza zoomed ahead, her toppings dancing with every move",
    "But, as the race progressed"
]

In [ ]:
## Checking if repeated sentences are labeled correctly
repeated_list = word_level_data['repeated'].tolist()
for sent in repeated_sentences:
  n = 0
  for i in range(len(word_level_data)):
    goldStandard = goldStandard_list[i]
#    n = 0
    if sent in goldStandard:
      n += 1
      print(goldStandard, id_list[i], int(sentence_id_list[i]))
      print(repeated_list[i])
  if n == 0:
    print('Did not find this entence')


In [ ]:
## Checking if segments are separated correctly, i.e., (5b)

separate_list = [("As Catsy sails, she meets a shy fish.",
                "Catsy says with a wide smile."), #“Join the ship,” Catsy says with a wide smile.

               ("You are my best friend, Penny, and I will always be here for you.", #Jane said, “You are my best friend, Penny, and I will always be here for you.”
                "And so, Jane and Penny lived happily, sharing many adventures together, day and night."),

               ("Even on rainy days, we can find joy!", #Joy said, “Even on rainy days, we can find joy!”
                "Roy nodded, happy to have such a joyful friend."),

               ("but Tara was there with her charm.",  #It wasn’t hard, but Tara was there with her charm.
                "she said, patting his arm gently with a pad"), #“Don’t worry, Jake, it’s just a little scar,” she said, patting his arm gently with a pad.

              ("She bandaged it fast and gave him a smile that could light up the dark.",
               "Jake beamed like a star as he darted back to class."), #“Thanks, Tara!” Jake beamed like a star as he darted back to class.

              ("Aunt Sue cheered", #Aunt Sue cheered, “Great catch, Patch!”
              "They ended the day with a treat, a yummy batch of cookies."),

              ("It was a sly owl perched on a branch.",
               "he hooted kindly."), #“Can I join?” he hooted kindly.

               ("Dale waved, hoping for a friend.",
                "Want to leap and play?", #“Hi, Dawn!” hooted Dale, “Want to leap and play?”
                "Dawn agreed, her eyes twinkling.")
               ]

In [ ]:
for pair in separate_list:
  for tok in pair:
    for i in range(len(word_level_data)):
      goldStandard = goldStandard_list[i]
      if tok in goldStandard:
        print(goldStandard, int(id_list[i]), int(sentence_id_list[i]))
  print('\n')

In [ ]:
## Checking if segments are expanded correctly, i.e. (6b)

expand_list = ["Roy called out, waving a corner of himself.",
               "Quick, we need to find a place to stay dry!",
               "Phew! We're safe here!",
               "Look at those colors!",
               "This is the best!",
               "Look at it soar!",
               "Look, a map!",
               "What a spot to teach my kittens to jump!",
               "This is fun!",
               "Go, cat!",
               "Would you like to play with us?",
               "search for shells!",
               "At the end, they said,",
               "Caroline laughed",
               "Can I join?",
               "Thanks, Sam!",
               "Hello, snail!",
               "Let's play!",
               "make a kite!",
               "His mother said",
               "Hurrah! My tooth came out!",
               "Come on, Zeus!", # Let’s play!",
               "At home, Zade patted Zeus and said",
               "It's in the box!",
               "What was that?",
               "This cake is so nice!",
               "Let's make it really tall!",
               "Help! My wing is stuck",
               "Where will we go first?"

]

In [ ]:
for tok in expand_list:
  n = 0
  for i in range(len(word_level_data)):
    goldStandard = goldStandard_list[i]
    if tok in goldStandard:
      try:
        print(goldStandard, int(id_list[i]), int(sentence_id_list[i]))
      except:
        print(goldStandard, int(id_list[i]))
      n += 1
  print('\n')

  if n == 0:
    print(tok, 'Did not find this sentence')

In [ ]:
## Checking if segments are combined correctly, i.e., (6c)

combine_list = [
    "Should we push on and find the hidden book?",
    "Look! The trees move in the wind!",
    "Shall we begin?",
    "Got you, Jake!",
    "This is the greatest of days!",
    "Wow, what a find!",
    "I won!",
    "Got you back!",
    "Let's go again!"
]

In [ ]:
for tok in combine_list:
  n = 0
  for i in range(len(word_level_data)):
    goldStandard = goldStandard_list[i]
    if tok in goldStandard:
      try:
        print(goldStandard, int(id_list[i]), int(sentence_id_list[i]))
      except:
        print(goldStandard, int(id_list[i]))
      n += 1
  print('\n')

  if n == 0:
    print(tok, 'Did not find this sentence')

**Word-level JSON file checks**

In [266]:
# ----------- Load JSON ----------
json_path = "export_157618_project-157618-at-2025-08-04-01-48-1a98679f.json"  # <- change to your actual path
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
## Looking at the JSON format

def print_structure_with_values(d, indent=0, max_string_length=80):
    prefix = "  " * indent
    if isinstance(d, dict):
        for k, v in d.items():
            if isinstance(v, (dict, list)):
                print(f"{prefix}- {k}:")
                print_structure_with_values(v, indent + 1)
            else:
                val_str = str(v)
                if len(val_str) > max_string_length:
                    val_str = val_str[:max_string_length] + "..."
                print(f"{prefix}- {k}: {val_str}")
    elif isinstance(d, list):
        if not d:
            print(f"{prefix}- []")
        else:
            for i, item in enumerate(d):
                if isinstance(item, (dict, list)):
                    print_structure_with_values(item, indent)
                else:
                    val_str = str(item)
                    if len(val_str) > max_string_length:
                        val_str = val_str[:max_string_length] + "..."
                    print(f"{prefix}- [{i}]: {val_str}")


# ---------- Print First Example ----------
print("🔍 Structure with example values from first item:\n")
#print_structure_with_values(data[0])
for item in data:
  task_id = item.get("id")
  if task_id == 185499485:
    print_structure_with_values(item)

In [ ]:
# Get different annotation types
types = []
from_names = []
type_from_name_pairs = []
for tok in data:
#  print(len(tok.get('annotations'))) # 1
  results = (tok.get('annotations')[0].get('result'))
  for result in results:
     result_type = result.get('type')
     types.append(result_type)
     from_name = result.get('from_name')
     from_names.append(from_name)
     type_from_name_pairs.append((result_type, from_name))

print(set(types))
print(set(from_names))
for tok in set(type_from_name_pairs):
  print(tok)

{'taxonomy', 'choices', 'labels', 'textarea'}
{'MixedErrorTaxonomy', 'spoken_word', 'issues', 'PhonologicalErrorType', 'WordAnnotation', 'mispronunciation_word', 'DisfluencyErrorType', 'StructuralErrorType', 'VisualTrackingErrorType', 'OrthographicErrorType', 'GrammaticalErrorType', 'spoken_words', 'produced_word'}
('choices', 'PhonologicalErrorType')
('taxonomy', 'MixedErrorTaxonomy')
('textarea', 'issues')
('labels', 'WordAnnotation')
('choices', 'GrammaticalErrorType')
('textarea', 'mispronunciation_word')
('textarea', 'spoken_words')
('choices', 'OrthographicErrorType')
('choices', 'DisfluencyErrorType')
('choices', 'VisualTrackingErrorType')
('textarea', 'spoken_word')
('textarea', 'produced_word')
('choices', 'StructuralErrorType')


In [382]:
## Doing checks and Output to CSV file
import csv
from collections import defaultdict


annotator_list = [
  "rose.m@ufl.edu",
  "emilianapulido@ufl.edu",
  "lamanjennifer@ufl.edu",
  "evanhalperin@ufl.edu",
  "claire.kuntz@ufl.edu",
  "ghineae@ufl.edu",
  "hamern@ufl.edu",
  "katherine.ball@ufl.edu"
]

annotator_map = {
  "rose.m@ufl.edu": "Maddie",
  "emilianapulido@ufl.edu": "Emi",
  "lamanjennifer@ufl.edu": "Jen",
  "evanhalperin@ufl.edu": "Evan",
  "claire.kuntz@ufl.edu": "Claire",
  "ghineae@ufl.edu": "Elizabeth",
  "hamern@ufl.edu": "Noah",
  "katherine.ball@ufl.edu": "Katherine"
}

error_dict = {}
for annotator in annotator_list:
  error_dict[annotator] = []

# -------- Extract & Format Data --------
rows = []
audio_dict = {} ## For comparing cross-annotations later

for item in data:
    task_id = item.get("id")


    annotations = item.get("annotations", [])
    if not annotations:
        continue
    
    goldStandard = item.get("data").get("goldStandard")
    if task_id == 185849965:
      goldStandard = "Pat had a pan."
    
    # Tasks that have more than one annotation:
    if len(annotations) > 1:
      if task_id not in audio_dict:
        audio_dict[task_id] = []
      for annotation in annotations:
        result = annotation.get("result", [])
        annotator = annotation.get("completed_by").get("email")    
        # Group all parts of a word using result.id
        grouped = defaultdict(lambda: {
        "task_id": task_id,
        "annotator": annotator,
        "start": None,
        "end": None,
        "intended_words": [],
        "produced_word": [],
        "IPA": [],
        "original_error_category": '',
        "error_category": '',
        "error_labels": [],
        "comments": [],
        "goldStandard": goldStandard
        })

        for r in result:
            result_id = r.get("id")
            value = r.get("value", {})
            r_type = r.get("type")
            from_name = r.get("from_name")

            if r_type == "labels" and from_name == "WordAnnotation":
                grouped[result_id]["start"] = value.get("start")
                grouped[result_id]["end"] = value.get("end")
                category = value.get("labels")[0]
                grouped[result_id]["error_category"] = category
                grouped[result_id]["original_error_category"] = category

            if grouped[result_id]["original_error_category"] != "Mixed Error" and r_type == "choices":
                category = grouped[result_id]["error_category"]
                if category != '' and category != "Mixed Error":
                    category = category.replace(' ', '')
                    if category in from_name:
                        labels = value.get("choices", [])
                        grouped[result_id]["error_labels"].extend(labels)

            if r_type == "textarea" and from_name == "spoken_words":
                category = grouped[result_id]["original_error_category"]
                if category in ["Run-on", "Contraction/Shortening"]:
            #  intended_words = ' '.join(value.get("text", []))
                    grouped[result_id]["intended_words"].extend(value.get("text", []))

            if r_type == "textarea" and from_name == "spoken_word":
                category = grouped[result_id]["original_error_category"]
                if category in [
                "Correct",
                "Phonological Error",
                "Orthographic Error",
                "Grammatical Error",
                "Structural Error",
                "Visual Tracking Error",
                "Disfluency Error",
                "Mixed Error",
                ] or "Mixed Error+" in category:
            #  intended_words = ' '.join(value.get("text", []))
                    grouped[result_id]["intended_words"].extend(value.get("text", []))

            if r_type == "textarea" and from_name == "produced_word":
                category = grouped[result_id]["original_error_category"]
                if category in [
                "Contraction/Shortening",
                "Visual Tracking Error",
                "Self Response",
                "Other",
                "Orthographic Error",
                "Grammatical Error",
                "Structural Error",
                "Mixed Error"
                ]:
            #  produced_word = ' '.join(value.get("text", []))
                    grouped[result_id]["produced_word"].extend(value.get("text", []))

            if r_type == "textarea" and from_name == "mispronunciation_word":
                category = grouped[result_id]["original_error_category"]
                if category in [
                "Contraction/Shortening",
                "Other",
            #  "Unintelligible",
                "Phonological Error",
                "Orthographic Error",
                "Grammatical Error",
                "Structural Error",
                "Visual Tracking Error",
                "Disfluency Error",
                "Mixed Error"
                ]:
                    grouped[result_id]["IPA"].extend(value.get("text", []))

            if r_type == "textarea" and from_name == "issues":
                grouped[result_id]["comments"].extend(value.get("text", []))

            ## e.g., "taxonomy": [["Disfluency Error", "Parental Aid"], ["Other"]]
            ## e.g., "taxonomy": [["Orthographic Sub.", "Phonological"], ["Phonological", "Consonant Substitution"]
            ## e.g., "taxonomy": [["Orthographic Sub.", "Phonological"], ["Phonological", "Consonant Substitution"], ["Phonological", "Consonant Omission"]
            
            if grouped[result_id]["original_error_category"] == "Mixed Error" and r_type == "taxonomy":
                category = grouped[result_id]["original_error_category"]
                if category == "Mixed Error":
                    labels = []
                    mixed = value.get("taxonomy", [])
                    for tok in mixed:
                        if tok[0] not in category:
                            category = category + '+' + tok[0]
                        if len(tok) > 1:
                            labels = labels + tok[1 : ]
                    grouped[result_id]["error_category"] = category
                    grouped[result_id]["error_labels"] = labels

        # Create a row for each word
        words = []
        for word_id, info in grouped.items():
            words.append({
                "task_id": task_id,
                "annotator": annotator,
                "start": info["start"],
                "end": info["end"],
                "intended_words": ' '.join(info["intended_words"]),
                "produced_word": ' '.join(info["produced_word"]),
                "IPA": info["IPA"],
                "error_category": info["error_category"],
                "error_labels": info["error_labels"],
                "comments": " | ".join(info["comments"]),
                "goldStandard": goldStandard
            })

        # Sort by start time
        sorted_segments = sorted(words, key=lambda x: (x["start"] is None, x["start"]))
        audio_dict[task_id].append(sorted_segments)

for k, v in audio_dict.items():
    len_list = [len(tok) for tok in v]
    max_len = max(len_list)
    for i in range(max_len):
        try:
            print(v[0][i])
            print(v[1][i])
            print('\n')
        except:
            try:
                print(v[0][i])
                print('\n')
            except:
                print(v[1][i])
                print('\n')
    print('\n')

{'task_id': 195798237, 'annotator': 'rose.m@ufl.edu', 'start': 0.27712351110370953, 'end': 0.657408008076859, 'intended_words': 'Zade', 'produced_word': '', 'IPA': [], 'error_category': 'Correct', 'error_labels': [], 'comments': '', 'goldStandard': 'Zade loved playing basketball.'}
{'task_id': 195798237, 'annotator': 'evanhalperin@ufl.edu', 'start': 0.27569512195121954, 'end': 0.6188475609756099, 'intended_words': 'Zade', 'produced_word': '', 'IPA': [], 'error_category': 'Correct', 'error_labels': [], 'comments': '', 'goldStandard': 'Zade loved playing basketball.'}


{'task_id': 195798237, 'annotator': 'rose.m@ufl.edu', 'start': 0.6613008718957262, 'end': 1.057867204299146, 'intended_words': 'loved', 'produced_word': '', 'IPA': [], 'error_category': 'Correct', 'error_labels': [], 'comments': '', 'goldStandard': 'Zade loved playing basketball.'}
{'task_id': 195798237, 'annotator': 'evanhalperin@ufl.edu', 'start': 0.6247134146341463, 'end': 1.0265243902439025, 'intended_words': 'loved',

In [ ]:
## Doing checks and Output to CSV file
import csv
from collections import defaultdict


annotator_list = [
  "rose.m@ufl.edu",
  "emilianapulido@ufl.edu",
  "lamanjennifer@ufl.edu",
  "evanhalperin@ufl.edu",
  "claire.kuntz@ufl.edu",
  "ghineae@ufl.edu",
  "hamern@ufl.edu",
  "katherine.ball@ufl.edu"
]

annotator_map = {
  "rose.m@ufl.edu": "Maddie",
  "emilianapulido@ufl.edu": "Emi",
  "lamanjennifer@ufl.edu": "Jen",
  "evanhalperin@ufl.edu": "Evan",
  "claire.kuntz@ufl.edu": "Claire",
  "ghineae@ufl.edu": "Elizabeth",
  "hamern@ufl.edu": "Noah",
  "katherine.ball@ufl.edu": "Katherine"
}

error_dict = {}
for annotator in annotator_list:
  error_dict[annotator] = []

# -------- Extract & Format Data --------
rows = []
audio_dict = {} ## For comparing cross-annotations later

for item in data:
    task_id = item.get("id")

    ## Checking to see which task has more than one annotation
    audio = item.get("data").get("audio")
    if task_id not in audio_dict:
      audio_dict[task_id] = 1
    else:
      audio_dict[task_id] += 1

    annotations = item.get("annotations", [])
    if not annotations:
        continue
    
    goldStandard = item.get("data").get("goldStandard")
    if task_id == 185849965:
      goldStandard = "Pat had a pan."
    result = annotations[0].get("result", [])
    annotator = annotations[0].get("completed_by").get("email")

    # Tasks that have more than one annotation:
    if len(annotations) > 1:
      print(task_id)
      
    # Group all parts of a word using result.id
    grouped = defaultdict(lambda: {
        "task_id": task_id,
        "annotator": annotator,
        "start": None,
        "end": None,
        "intended_words": [],
        "produced_word": [],
        "IPA": [],
        "original_error_category": '',
        "error_category": '',
        "error_labels": [],
        "comments": [],
        "goldStandard": goldStandard
    })

    for r in result:
        result_id = r.get("id")
        value = r.get("value", {})
        r_type = r.get("type")
        from_name = r.get("from_name")

        if r_type == "labels" and from_name == "WordAnnotation":
            grouped[result_id]["start"] = value.get("start")
            grouped[result_id]["end"] = value.get("end")
            category = value.get("labels")[0]
            grouped[result_id]["error_category"] = category
            grouped[result_id]["original_error_category"] = category

        if grouped[result_id]["original_error_category"] != "Mixed Error" and r_type == "choices":
          category = grouped[result_id]["error_category"]
          if category != '' and category != "Mixed Error":
            category = category.replace(' ', '')
            if category in from_name:
              labels = value.get("choices", [])
              grouped[result_id]["error_labels"].extend(labels)

        if r_type == "textarea" and from_name == "spoken_words":
          category = grouped[result_id]["original_error_category"]
          if category in ["Run-on", "Contraction/Shortening"]:
          #  intended_words = ' '.join(value.get("text", []))
            grouped[result_id]["intended_words"].extend(value.get("text", []))

        if r_type == "textarea" and from_name == "spoken_word":
          category = grouped[result_id]["original_error_category"]
          if category in [
            "Correct",
            "Phonological Error",
            "Orthographic Error",
            "Grammatical Error",
            "Structural Error",
            "Visual Tracking Error",
            "Disfluency Error",
            "Mixed Error",
          ] or "Mixed Error+" in category:
          #  intended_words = ' '.join(value.get("text", []))
            grouped[result_id]["intended_words"].extend(value.get("text", []))

        if r_type == "textarea" and from_name == "produced_word":
          category = grouped[result_id]["original_error_category"]
          if category in [
            "Contraction/Shortening",
            "Visual Tracking Error",
            "Self Response",
            "Other",
            "Orthographic Error",
            "Grammatical Error",
            "Structural Error",
            "Mixed Error"
          ]:
          #  produced_word = ' '.join(value.get("text", []))
            grouped[result_id]["produced_word"].extend(value.get("text", []))

        if r_type == "textarea" and from_name == "mispronunciation_word":
          category = grouped[result_id]["original_error_category"]
          if category in [
            "Contraction/Shortening",
            "Other",
          #  "Unintelligible",
            "Phonological Error",
            "Orthographic Error",
            "Grammatical Error",
            "Structural Error",
            "Visual Tracking Error",
            "Disfluency Error",
            "Mixed Error"
          ]:
            grouped[result_id]["IPA"].extend(value.get("text", []))

        if r_type == "textarea" and from_name == "issues":
            grouped[result_id]["comments"].extend(value.get("text", []))

        ## e.g., "taxonomy": [["Disfluency Error", "Parental Aid"], ["Other"]]
        ## e.g., "taxonomy": [["Orthographic Sub.", "Phonological"], ["Phonological", "Consonant Substitution"]
        ## e.g., "taxonomy": [["Orthographic Sub.", "Phonological"], ["Phonological", "Consonant Substitution"], ["Phonological", "Consonant Omission"]
        
        if grouped[result_id]["original_error_category"] == "Mixed Error" and r_type == "taxonomy":
          category = grouped[result_id]["original_error_category"]
          if category == "Mixed Error":
            labels = []
            mixed = value.get("taxonomy", [])
            for tok in mixed:
              if tok[0] not in category:
                category = category + '+' + tok[0]
              if len(tok) > 1:
                labels = labels + tok[1 : ]
            grouped[result_id]["error_category"] = category
            grouped[result_id]["error_labels"] = labels

    # Create a row for each word
    words = []
    for word_id, info in grouped.items():
        words.append({
            "task_id": task_id,
            "annotator": annotator,
            "start": info["start"],
            "end": info["end"],
            "intended_words": ' '.join(info["intended_words"]),
            "produced_word": ' '.join(info["produced_word"]),
            "IPA": info["IPA"],
            "error_category": info["error_category"],
            "error_labels": info["error_labels"],
            "comments": " | ".join(info["comments"]),
            "goldStandard": goldStandard
        })

    # Sort by start time
    sorted_segments = sorted(words, key=lambda x: (x["start"] is None, x["start"]))
    rows.extend(sorted_segments)

    ## -------- Checking for segmentation overlap --------
    overlap_found = False
    for i in range(len(sorted_segments) - 1):
      task_id = sorted_segments[i]["task_id"]
      annotator = sorted_segments[i]["annotator"]
      current_info = sorted_segments[i]
      next_info = sorted_segments[i+1]
      if current_info["end"] is not None and next_info["start"] is not None:
          if current_info["end"] > next_info["start"]:
          #    print(f"{task_id} {annotator} - Overlap detected between word segments {int(i) + 1} and {int(i) + 2}:")
          #    print(f"   current end = {current_info['end']:.4f} >= next start = {next_info['start']:.4f}")
          #    print(current_info, next_info)
          #    print('\n')
              error_dict[annotator].append([task_id, annotator, "Overlap detected between the following", current_info, next_info, '\n'])
              overlap_found = True


# -------- Write CSV --------
csv_path = "formatted_annotations_with_comments.csv"
task_intended_words_dict = {}

with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["task_id", "annotator", "start", "end", "intended_words", "produced_word", "IPA", "error_category", "error_labels", "comments", "goldStandard"])
    writer.writeheader()
    for row in rows:
        writer.writerow(row)
        comments = row.get("comments")
        start = row.get("start")
        end = row.get("end")
        ## Doing checks
     #   if comments == '': # for each task, if there are comments, comments together are assigned a new id, and there are no other annotations under this id
        if start is not None and end is not None:
          task_id = row.get("task_id")      
          annotator = row.get("annotator")
          intended_words = row.get("intended_words")
          produced_word = row.get("produced_word")
          IPA = row.get("IPA")
          category = row.get("error_category")
          labels = row.get("error_labels")
          goldStandard = row.get("goldStandard")

          # Keeping track of all intended words written by annotators to identify if anything is misspelled
          if task_id not in task_intended_words_dict:
            task_intended_words_dict[task_id] = [intended_words]
          else:
            task_intended_words_dict[task_id].append(intended_words)

          new_row = row
          del new_row["task_id"]
          del new_row["annotator"]
          del new_row["start"]
          del new_row["end"]
          del new_row["comments"]

          ## Checking to see if annotators misspelled any word
          if 'Contraction' in category and intended_words not in goldStandard:
            error_dict[annotator].append([task_id, annotator, "Intended word(s) might be misspelled", new_row])
          if 'Run-on' in category and ' ' in intended_words and intended_words not in goldStandard:
            error_dict[annotator].append([task_id, annotator, "Intended word(s) might be misspelled", new_row])
          if 'Contraction' not in category and 'Run-on' not in category:
            if intended_words not in goldStandard.split():
              error_dict[annotator].append([task_id, annotator, "Intended word(s) might be misspelled", new_row])
            else:
              if intended_words[0].isupper() is False and intended_words.capitalize() in goldStandard.split():
                if intended_words.capitalize() not in task_intended_words_dict[task_id]:
                  if [task_id, annotator, "NEW Intended word(s) might be misspelled", comments, new_row] not in error_dict[annotator]:
                    error_dict[annotator].append([task_id, annotator, "NEW Intended word(s) might be misspelled", comments, new_row])

          if produced_word == "NONE":
          #  print(task_id, annotator, new_row)
            error_dict[annotator].append([task_id, annotator, "Don't write NONE as Produced Word", new_row])
          #    pass      

          if category == '':
          #   pass
          #  print(task_id, annotator, "Missing Error Category", new_row)
            error_dict[annotator].append([task_id, annotator, "Missing Error Category", new_row])
         
          if category not in ["Correct", "Contraction/Shortening", "Unintelligible", "Whispering", "Other", "Self Response", "Run-on"] and len(labels) == 0:
            if "Mixed Error" not in category:
            #   pass
            #  print(task_id, annotator, "Missing choosing Error Type(s)", new_row)
              error_dict[annotator].append([task_id, annotator, "Missing choosing Error Type(s)", new_row])  
            else:
               exclude_count = 1
               category_list = category.split('+')
               for c in ["Correct", "Contraction/Shortening", "Unintelligible", "Whispering", "Other", "Self Response", "Run-on"]:
                  if c in category_list or c in category:
                    exclude_count += 1
               if exclude_count != len(category_list):
                #  pass
                #  print(task_id, annotator, "Missing choosing Error Type(s)", new_row)
                  error_dict[annotator].append([task_id, annotator, "Missing choosing Error Type(s)", new_row])

          if 'Run-on' in category or 'Run-on' in labels:
            if ' ' in intended_words:
            #  pass
            #  print(task_id, annotator, "The run-on word needs to be segmented if possible", new_row)
              error_dict[annotator].append([task_id, annotator, "The run-on word needs to be segmented if possible", new_row])

          if "Mixed Error" in category and len(category.split("+")) <= 1:
          #  pass
          #  print(task_id, annotator, "Only one error label was found for Mixed Error", new_row)
            error_dict[annotator].append([task_id, annotator, "Only one error label was found for Mixed Error", new_row]) 

          ### Checking Maddie's previous annotations
          if "Prolongation" in labels: # and category != "Disfluency Error":
            # Maddie's previous annotations
            # IPA: : --> Consistent with guideline
            # Produced Word: Repetition of the prolonged sound
            try:
              if ":" not in IPA[0] and 'ː' not in IPA[0]:
              #  print(task_id, annotator, "Might be missing : for IPA for Prolongation", new_row)
                error_dict[annotator].append([task_id, annotator, "Might be missing : for IPA for Prolongation", new_row]) 
            except:
              pass
            #  error_dict[annotator].append([task_id, annotator, "Might be missing IPA", new_row]) 
            
            if produced_word != '':
              if ":" not in produced_word and 'ː' not in produced_word:
              #  print(task_id, annotator, "Might need to add : for Produced Word for Prolongation", new_row)
                error_dict[annotator].append([task_id, annotator, "Might need to add : for Produced Word for Prolongation", new_row]) 

          if "Stutter" in labels:
            # Maddie's annotations
            # IPA: . --> Consistent with guideline
            # Produced Word: A repetition of the stuttered part and three dots … (Ex: h…hi)
            try:
              if "." not in IPA[0]:
              #  print(task_id, annotator, "Might be missing . for IPA for Stutter", new_row)
                error_dict[annotator].append([task_id, annotator, "Might be missing . for IPA for Stutter", new_row]) 
            except:
              pass
            #  error_dict[annotator].append([task_id, annotator, "Might be missing IPA", new_row]) 
            
            if produced_word != '':               
                if "." not in produced_word:
              #     print(task_id, annotator, "Might be missing . for Produced Word for Stutter", new_row)
                   error_dict[annotator].append([task_id, annotator, "Might be missing . for Produced Word for Stutter", new_row]) 

          if "Broken Word" in labels:
            # Maddie's previous annotations:    
            # IPA: .
            # Produced Word: three dots …
            if "rose" in annotator and "." in intended_words:
            #  print(task_id, annotator, "Maddie might need to revisit Intended Word for Broken Word", new_row)  
              error_dict[annotator].append([task_id, annotator, "Maddie might need to revisit Intended Word for Broken Word", new_row])                 
            if "rose" in annotator and "." not in IPA[0] and "|" not in IPA[0]:
            #  print(task_id, annotator, "Maddie might need to revisit IPA for Broken Word", new_row)  
              error_dict[annotator].append([task_id, annotator, "Maddie might need to revisit IPA for Broken Word", new_row])  
            if "rose" in annotator and "." in IPA[0] and "Stutter" in labels:
              error_dict[annotator].append([task_id, annotator, "Maddie might need to revisit IPA for Broken Word and Stutter", new_row])  
            try:
              if "rose" not in annotator and "|" not in IPA[0]:             
              #  print(task_id, annotator, "Might be missing | for IPA for Broken Word", new_row)  
                error_dict[annotator].append([task_id, annotator, "Might be missing | for IPA for Broken Word", new_row])     
            except:
              error_dict[annotator].append([task_id, annotator, "Might be missing IPA", new_row])     
              
            if produced_word != '':
              if "rose" in annotator and "." not in produced_word and "|" not in produced_word:
              #  print(task_id, annotator, "Maddie might need to revisit Produced Word for Broken Word", new_row)
                error_dict[annotator].append([task_id, annotator, "Maddie might need to revisit Produced Word for Broken Word", new_row])  
              if "rose" in annotator and "." in produced_word and "Stutter" in labels:
                error_dict[annotator].append([task_id, annotator, "Maddie might need to revisit Produced Word for Broken Word and Stutter", new_row])  
              
              try:
                if "rose" not in annotator and "|" not in produced_word:             
                #  print(task_id, annotator, "Might be missing | for Produced Word for Broken Word", new_row)
                  error_dict[annotator].append([task_id, annotator, "Might be missing | for Produced Word for Broken Word", new_row])  
              except:
                error_dict[annotator].append([task_id, annotator, "Might be missing Produced Word", new_row])  
               
          if "Visual" in category:
             error_dict[annotator].append([task_id, annotator, "Check your annotations for Visual Tracking Errors", new_row])  
    
          if IPA == []:
            if "Contraction/Shortening" in category or "Phonological Error" in category or "Orthographic Error" in category or "Grammatical Error" in category or (category == "Structural Error" and "Word Insertion" in labels) or (category == "Disfluency Error" and "Stutter" in labels) or (category == "Disfluency Error" and "Interjection" in labels) or (category == "Disfluency Error" and "Prolongation" in labels) or (category == "Disfluency Error" and "Broken Word" in labels):
            #  print(task_id, annotator, "Missing IPA", row)
              error_dict[annotator].append([task_id, annotator, "Missing IPA", new_row])
  
          if intended_words == '' and produced_word == '' and IPA == []:
            if labels != ["Unfilled Pause"]:
              if category not in ["Unintelligible", "Whispering", "Other"] and "Mixed Error" not in category:
                if not (category == "Disfluency Error" and labels == []):
                #  print(task_id, annotator, "Need to provide relevant annotations for this word", new_row)
                  error_dict[annotator].append([task_id, annotator, "Need to provide relevant annotations for this word; especially if the Error Type contains Parental Aid, make sure to re-consult the guideline", new_row]) 

              elif "Mixed Error" in category:
                exclude_count = 1
                category_list = category.split('+')
                for c in ["Unintelligible", "Whispering", "Other"]:
                  if c in category_list or c in category:
                    exclude_count += 1
                if exclude_count != len(category_list):
                #  print(task_id, annotator, "Need to provide relevant annotations for this word", new_row, exclude_count)
                  error_dict[annotator].append([task_id, annotator, "Need to provide relevant annotations for this word; especially if the Error Type contains Parental Aid, make sure to re-consult the guideline", new_row])

          if not (intended_words == '' and produced_word == '' and IPA == []):
            if labels == ["Parental Aid"] and produced_word == '':
              try:
                assert intended_words != ''
              except:
                error_dict[annotator].append([task_id, annotator, "Consult the guideline about your annotations for Parental Aid and Error Category", new_row])

          if not (intended_words == '' and produced_word == '' and IPA == []):
            if intended_words == '':
              if labels == ["Parental Aid"]: 
                try:
                  assert "Mixed Error" in category and "Other" in category
                  assert produced_word != ''
                except:
                  error_dict[annotator].append([task_id, annotator, "Consult the guideline about your annotations for Parental Aid and Error Category", new_row])

              elif labels != ['Unfilled Pause'] and labels != ["Interjection"] and "Word Insertion" not in labels:
                exclude = ["Whispering", "Other", "Unintelligible", "Self Response"]
                if category not in exclude:
                  if "Mixed Error" in category:
                    exclude_count = 1
                    category_list = category.split('+')
                    for c in exclude:
                      if c in category_list:
                        exclude_count += 1
                    if exclude_count != len(category_list):
                      #  if produced_word == '':
                      intended_word_check = 'not pass'
                    #  print(task_id, annotator, "Missing intended word(s)", new_row)
                      error_dict[annotator].append([task_id, annotator, "Missing intended word(s)", new_row, exclude_count])

                  else:
                  #  if produced_word == '':
                    intended_word_check = 'not pass'
                  #  print(task_id, annotator, "Missing intended word(s)", new_row)
                    error_dict[annotator].append([task_id, annotator, "Missing intended word(s)", new_row])

          if ("Contraction/Shortening" in category or "Visual Tracking Error" in category or "Self Response" in category or "Orthographic Error" in category or "Grammatical Error" in category or "Structural Error" in category or "Other" in category):
          #  if "Mixed Error" not in category:
            if produced_word == '':
            #  print(task_id, annotator, "Missing produced word(s)", new_row)
              error_dict[annotator].append([task_id, annotator, "Missing produced word(s)", new_row])

        #  if "Disfluency" in category and "Unfilled Pause" not in labels and labels != [] and labels != ["Interjection"] and intended_words == '' and produced_word == '':
        #    print("CHECK ME", task_id, annotator, new_row)
          
        #  if "Other" in category and intended_words == '' and produced_word == '':
        #    print(task_id, annotator, "CHECK OTHER", new_row)

          if "Unfilled Pause" in labels and intended_words != '':
            error_dict[annotator].append([task_id, annotator, "There should not be Intended Word filled out for Unfilled Pause", new_row])

          if "Word Insertion" in labels and intended_words != '':
            error_dict[annotator].append([task_id, annotator, "There should not be Intended Word filled out for Word Insertion", new_row])

          if "Interjection" in labels and intended_words != '':
            error_dict[annotator].append([task_id, annotator, "There should not be Intended Word filled out for Interjection", new_row])

print(f"✅ Done! Output saved to: {csv_path}")

for annotator in annotator_list:
  with open(annotator_map[annotator] + '_fixes.txt', 'w') as f:
    annotation_errors = error_dict[annotator]
    try:
      sorted_annotation_errors = sorted(annotation_errors, key=lambda x: x[0])
    except:
      print(sorted_annotation_errors)

    for error in sorted_annotation_errors:
      f.write(' '.join(str(tok) for tok in error) + '\n')
  
  f.close()


195798237
195798276
195798278
195798279
195798280
195798322
195798388
195798392
195798443
195798528
195798555
195798619
195798679
195798694
195798701
195798739
195798764
195798870
195798871
195798872
195798873
195798874
195798875
195798876
195798877
195798878
195798879
195798880
195798881
195798882
195799008
195799087
195799124
195799177
195799189
195799242
✅ Done! Output saved to: formatted_annotations_with_comments.csv


In [ ]:
### Checking cross-annotations; JSON file seems to only save the latest annotations (from two annotators)
import pandas as pd

csv_path = pd.read_csv("export_157618_project-157618-at-2025-08-04-03-01-f5029ec4.csv")
word_annotation_list = csv_path['WordAnnotation'].tolist()
id_list = csv_path['id'].tolist()
id_annotations_dict = {}
for i in range(len(id_list)):
    idx = id_list[i]
    annotation = json.loads(word_annotation_list[i])
    annotation_list = []
    for tok in annotation:
        for k, v in tok.items():
            annotation_list.append(k + ' ' + str(v))
    if idx not in id_annotations_dict:       
        id_annotations_dict[idx] = [annotation_list]
    else:
        id_annotations_dict[idx].append(annotation_list)

overall_num_segment_agreement = []
overall_error_category_agreement = []
overall_error_label_agreement = []

for idx in list(set(id_list)):
    if id_list.count(idx) > 1:
        print(idx)
        max_len = []
        for tok in id_annotations_dict[idx]:
            tok_len = len(tok)
            max_len.append(tok_len)
        max_len = max(max_len)
        annotation1 = id_annotations_dict[idx][0]
        new_annotation1 = []
        for ann in annotation1:
            if 'Unfilled Pause' not in ann:
                new_annotation1.append(ann)

        annotation2 = id_annotations_dict[idx][1]
        new_annotation2 = []
        for ann in annotation2:
            if 'Unfilled Pause' not in ann:
                new_annotation2.append(ann)

        if len(new_annotation1) == len(new_annotation2):
            overall_num_segment_agreement.append(1)
        else:
            overall_num_segment_agreement.append(0)
            
        for i in range(max_len):
            try:
                print(new_annotation1[i], new_annotation2[i])
            except:
                try:
                    print('NONE', new_annotation2[i])
                except:
                    print(new_annotation1[i], 'NONE')
    #    for tok in id_annotations_dict[idx]:
    #    print(tok)



195799087
end 1.230368166351607 end 1.2322106977950358
start 1.0026035831758036 start 1.0047706947212132
labels ['Correct'] labels ['Phonological Error']
channel 0 channel 0
end 1.463991645557656 end 1.4954424236441899
start 1.2318328903591682 start 1.2333495445655573
labels ['Run-on'] labels ['Run-on']
channel 0 channel 0
end 2.0630637646502836 end 2.045821240472578
start 1.4661887315689983 start 1.4967439798329845
labels ['Run-on'] labels ['Correct']
channel 0 channel 0
end 2.352346756143667 end 2.348749119348664
start 2.065260850661626 start 2.047610746180105
labels ['Run-on'] labels ['Correct']
channel 0 channel 0
end 2.5200576550094516 end 2.5195732751494844
start 2.354543842155009 start 2.350213444534428
labels ['Correct'] labels ['Correct']
channel 0 channel 0
end 2.8298467826086955 end 2.846091219476496
start 2.5215223790170134 start 2.521525549853981
labels ['Correct'] labels ['Correct']
channel 0 channel 0
end 3.217998644612477 end 3.2261339491801873
start 2.8466911086956523 

**Sentence-level checks**

In [ ]:
## Reading data
original_data = pd.read_csv('export_157513_project-157513-at-2025-07-01-07-44-30c2d71c.csv', delimiter = '\t')
original_data.columns
original_data.head(n=8)
original_id_list = original_data['id'].tolist()

## Filter out cases with no annotations for sentence segmentation (problematic cases)
data = original_data[~(original_data['SentenceLabel'].isna())]

In [ ]:
## Cross-annotated tasks will appear the number of times that correspond to how many times it's been annotated
print(len(original_data), len(data))
print('\n')
for id in original_id_list:
  if id not in id_list:
    print(id)

339 323


188964230
188964235
188964256
188964266
188964274
188964303
188964313
188964317
190669668
190669691
190669734
190669755
190669767
190669779
190669830
190669836


In [ ]:
## Loading some variables
SentenceLabel_list = data['SentenceLabel'].tolist() ## Timestamps/Segmentations
SentenceSelect_list = data['SentenceSelect'].tolist() ## Annotated intended sentences
goldStandardText_list = data['goldStandardText'].tolist() ## Original gold standard texts from story.xlsx
annotator_list = data['annotator'].tolist() ## Annotators
id_list = data['id'].tolist() ## Audio ID showing up on the UI
possibleSentences_list = data['possibleSentences'].tolist() ## Use this as the most updated reference sentence segmentation used as the Intended Sentence in the UI (the list where the annotators chose from)
issues_list = data['issues'].tolist() # Comments from annotators
matching_file_list = data['matching_file'].tolist()

In [ ]:
## Generate a list of audio files with no annotations
original_id_list = original_data['id'].tolist()
unannotated_audios = list(set(original_id_list) - set(id_list))

In [ ]:
## Do a sample check
print("Example check")
print("Timstamps: ", SentenceLabel_list[0])
print("Intended sentences: ", SentenceSelect_list[0])
print("Goldstandard sentences: ", goldStandardText_list[0])

Example check
Timstamps:  [{"end":18.345426558896467,"start":3.1734557118402047,"labels":["Sentence"],"channel":0},{"end":39.2403147289904,"start":22.41030241451201,"labels":["Sentence"],"channel":0},{"end":55.682110145675125,"start":41.411180314289275,"labels":["Sentence"],"channel":0},{"end":79.07180958205363,"start":57.02277979598388,"labels":["Sentence"],"channel":0},{"end":99.07659514926412,"start":79.39866705878178,"labels":["Sentence"],"channel":0},{"end":110.16,"start":99.53641159957664,"labels":["Sentence"],"channel":0}]
Intended sentences:  ["Pat had a pan.","Pat and the pan sat.","Pat and the pan fit.","Pat tapped the pan.","The pan slid.","Pat grinned."]
Goldstandard sentences:  1. Pat had a pan.
2. Pat and the pan sat.
3. Pat and the pan fit.
4. Pat tapped the pan.
5. The pan slid.
6. Pat grinned.


In [ ]:
for i in range(len(data)):

  ## Annotated time stamps
  SentenceLabel_str = ''
  try:
    SentenceLabel_str = json.loads(SentenceLabel_list[i])
  except:
    SentenceLabel_str = ast.literal_eval(SentenceLabel_list[i])

  ## Sort timestamps by start time
#  timestamps_sorted = sorted(SentenceLabel_str, key=lambda x: float(x['start']))

  ## Annotated Intended Sentences
  selected_sentences = ''
  try:
    selected_sentences = json.loads(SentenceSelect_list[i])
  except:
    try:
      selected_sentences = ast.literal_eval(SentenceSelect_list[i])
    except:
      ## When an audio has just one utterance, its intended sentence is returned as a single string, not a list with one string
      selected_sentences = [SentenceSelect_list[i]]

    # Sorting timestamps and annotated intended sentences together (ignore "Other")
  combined = [
    (ts, s) for ts, s in zip(SentenceLabel_str, selected_sentences) # This works even when lengths of timestamps and annotated intended sentences are different; therefore this step is after the Check above
    if s != "Other"
  ]

  # Sort by start time
  combined_sorted = sorted(combined, key=lambda x: float(x[0]["start"]))
  timestamps_sorted = [ts for ts, _ in combined_sorted]
  selected_sentences_sorted = [s for _, s in combined_sorted]

  ## Original old standard sentences
  goldStandardText_str = goldStandardText_list[i]
  gold_sentences = [line.split('. ', 1)[1] for line in goldStandardText_str.strip().split('\n')]

  ## Actual list of sentence segmentation used in the UI
  possibleSentence_str = json.loads(possibleSentences_list[i])
  possible_sentences = [item["value"] for item in possibleSentence_str][ : -1] # Remove 'Other'

  ## Checking if any segmentation does not have an Intended Sentence; this includes cases such as 'Other'
  ## If so, the number of timestamps would not match the number of intended sentences
  if len(SentenceLabel_str) != len(selected_sentences):
    print(id_list[i], len(SentenceLabel_str), len(selected_sentences), "Some segmentation does not have an Intended Sentence")
    print(possible_sentences)
    print(selected_sentences_sorted)
    print('\n')

In [ ]:
for i in range(len(data)):

  ## Annotated time stamps
  SentenceLabel_str = ''
  try:
    SentenceLabel_str = json.loads(SentenceLabel_list[i])
  except:
    SentenceLabel_str = ast.literal_eval(SentenceLabel_list[i])

  ## Sort timestamps by start time
#  timestamps_sorted = sorted(SentenceLabel_str, key=lambda x: float(x['start']))

  ## Annotated Intended Sentences
  selected_sentences = ''
  try:
    selected_sentences = json.loads(SentenceSelect_list[i])
  except:
    try:
      selected_sentences = ast.literal_eval(SentenceSelect_list[i])
    except:
      ## When an audio has just one utterance, its intended sentence is returned as a single string, not a list with one string
      selected_sentences = [SentenceSelect_list[i]]

  ## Checking if there are any duplicates within a list of intended sentences
  ## There are two possible case scenarios:
  ## (1) A sentence was repeated in child production
  ## (2) Identify cases with the first type of sentence segmentation error where the sentence should be split into two
  ## RESULTS thus far: Additionally idiosyncratic case: 190669709 Sentence segmentation error with the first listed intended sentence. It contains the title of the story, which is not read. Further, the title is accidentally combined with the first sentence of the story, which the child also does not read. "Title: Minnie's Spring Morning Minnie woke up early one sunny spring morning."
  temp_selected_sentences = [item for item in selected_sentences if item != 'Other']
  try:
    assert len(temp_selected_sentences) == len(set(temp_selected_sentences))
  except:
    print(id_list[i], len(temp_selected_sentences), len(set(temp_selected_sentences)))
    for sent in list(set(temp_selected_sentences)):
      if temp_selected_sentences.count(sent) != 1:
        print(sent)
        print('\n')
    print('\n')

188964183 3 2
Once upon a time, a mouse named Lace found a nice spot to dance.




188964194 12 10
One day, Whiskers found a hat on the mat.


The hat was big and flat.




188964207 7 6
Roy was sad but then spotted it on the shore.




188964221 10 8
They cheered as he got faster at each turn.


In a test of speed, he raced against the tallest kids.




188964227 6 5
He spies a bright site by the pine.




188964234 12 11
As Catsy sails, she meets a shy fish. “Join the ship,” Catsy says with a wide smile.




188964237 14 13
Jane said, “You are my best friend, Penny, and I will always be here for you.” And so, Jane and Penny lived happily, sharing many adventures together, day and night.




188964269 10 9
Joy said, “Even on rainy days, we can find joy!” Roy nodded, happy to have such a joyful friend.




188964287 9 7
She bandaged it fast and gave him a smile that could light up the dark.  “Thanks, Tara!” Jake beamed like a star as he darted back to class.


It wasn’t hard, but Tara 

In [ ]:
for i in range(len(data)):

  ## Annotated time stamps
  SentenceLabel_str = ''
  try:
    SentenceLabel_str = json.loads(SentenceLabel_list[i])
  except:
    SentenceLabel_str = ast.literal_eval(SentenceLabel_list[i])

  ## Sort timestamps by start time
#  timestamps_sorted = sorted(SentenceLabel_str, key=lambda x: float(x['start']))

  ## Annotated Intended Sentences
  selected_sentences = ''
  try:
    selected_sentences = json.loads(SentenceSelect_list[i])
  except:
    try:
      selected_sentences = ast.literal_eval(SentenceSelect_list[i])
    except:
      ## When an audio has just one utterance, its intended sentence is returned as a single string, not a list with one string
      selected_sentences = [SentenceSelect_list[i]]

  # Sorting timestamps and annotated intended sentences together (ignore "Other")
  combined = [
    (ts, s) for ts, s in zip(SentenceLabel_str, selected_sentences) # This works even when lengths of timestamps and annotated intended sentences are different; therefore this step is after the Check above
    if s != "Other"
  ]

  # Sort by start time
  combined_sorted = sorted(combined, key=lambda x: float(x[0]["start"]))
  timestamps_sorted = [ts for ts, _ in combined_sorted]
  selected_sentences_sorted = [s for _, s in combined_sorted]

  ## Original old standard sentences
  goldStandardText_str = goldStandardText_list[i]
  gold_sentences = [line.split('. ', 1)[1] for line in goldStandardText_str.strip().split('\n')]

  ## Actual list of sentence segmentation used in the UI
  possibleSentence_str = json.loads(possibleSentences_list[i])
  possible_sentences = [item["value"] for item in possibleSentence_str][ : -1] # Remove 'Other'

  ## Checking if child reverses any sentence order in production, regardless of whether they produced all sentences from the Intended Sentence list
  index_list = []
  temp_selected_sentences_sorted = []
  for z in range(len(selected_sentences_sorted)):
    sent = selected_sentences_sorted[z]
    if sent not in temp_selected_sentences_sorted:
      temp_selected_sentences_sorted.append(sent)

  for z in range(len(selected_sentences_sorted)):
    index_list.append(possible_sentences.index(selected_sentences_sorted[z]))

  new_index_list = sorted(index_list)
  if new_index_list != index_list:
    print(id_list[i], "Child reverses sentence order", matching_file_list[i])
    print(index_list)
    print(new_index_list)
    print('\n')

188964183 Child reverses sentence order uid_gUfM3yODstSdwltidtEzSlIo0q93_sid_YJYpGQOGORLj0onYXX3W_1742770181.wav
[0, 1, 0]
[0, 0, 1]


188964194 Child reverses sentence order uid_4w6SC0KmlKV4YmJTalgWiZalQHi2_sid_YEGwiBRYmGj8UEfBKjEL_1742332995.wav
[0, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9]


188964221 Child reverses sentence order uid_FUCVnaPJeyV5kzao6jmObtdzLUE2_sid_VP8ZqGQAw050ePA4KqWb_1742904863.wav
[0, 1, 2, 3, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 2, 3, 3, 4, 5, 6, 7]


188964252 Child reverses sentence order uid_H7rAk5StNLc0VysmHcZQZTQFpBx2_sid_1BvqAU3Y1VCwhTpvVr78_1741207795.wav
[0, 1, 2, 3, 4, 5, 6, 7, 12, 9, 10, 11, 12, 13, 14, 16, 17, 19, 20, 21, 23, 24, 25, 26, 27]
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 12, 13, 14, 16, 17, 19, 20, 21, 23, 24, 25, 26, 27]


188964312 Child reverses sentence order uid_M3RVTGHWrAMCChSiCDrpejluqjQ2_sid_J3DTSl0s0xBwSzdlZEZW_1742137192.wav
[0, 1, 2, 3, 4, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23

In [ ]:
## cases identified by Michael but was not in my list; they actually do not have reverse ordering
specific_cases = ['uid_HGLxgWVfJZWhAsqMf761d40h3eQ2_sid_DmEr4pqsDFri8xozTdsS_1743276724.wav',
'uid_gUfM3yODstSdwltidtEzSlIo0q93_sid_yYkurtcTpL0BYwYKWgIi_1742427844.wav',
'uid_ovpeape9PZbAq7VLrirwGOYrvsh1_sid_vrmxG2Pc3NPfDSVnUg5r_1742390017.wav',
'uid_HGLxgWVfJZWhAsqMf761d40h3eQ2_sid_F4tWmApCfflaPRfdT4xX_1741271551.wav',
'uid_Gt0cspsUtRSvEXXHZKUWlVAfyVq2_sid_qKQmMKuXG2FOUvLlR8Nk_1742062228.wav'
]

for tok in matching_file_list:
  if tok in specific_cases:
    index = matching_file_list.index(tok)
    print(id_list[index])

188964196
188964215
190669735
190669738
190669747


In [ ]:
for i in range(len(data)):

  ## Annotated time stamps
  SentenceLabel_str = ''
  try:
    SentenceLabel_str = json.loads(SentenceLabel_list[i])
  except:
    SentenceLabel_str = ast.literal_eval(SentenceLabel_list[i])

  ## Sort timestamps by start time
#  timestamps_sorted = sorted(SentenceLabel_str, key=lambda x: float(x['start']))

  ## Annotated Intended Sentences
  selected_sentences = ''
  try:
    selected_sentences = json.loads(SentenceSelect_list[i])
  except:
    try:
      selected_sentences = ast.literal_eval(SentenceSelect_list[i])
    except:
      ## When an audio has just one utterance, its intended sentence is returned as a single string, not a list with one string
      selected_sentences = [SentenceSelect_list[i]]

  # Sorting timestamps and annotated intended sentences together (ignore "Other")
  combined = [
    (ts, s) for ts, s in zip(SentenceLabel_str, selected_sentences) # This works even when lengths of timestamps and annotated intended sentences are different; therefore this step is after the Check above
    if s != "Other"
  ]

  # Sort by start time
  combined_sorted = sorted(combined, key=lambda x: float(x[0]["start"]))
  timestamps_sorted = [ts for ts, _ in combined_sorted]
  selected_sentences_sorted = [s for _, s in combined_sorted]

  ## Original old standard sentences
  goldStandardText_str = goldStandardText_list[i]
  gold_sentences = [line.split('. ', 1)[1] for line in goldStandardText_str.strip().split('\n')]

  ## Actual list of sentence segmentation used in the UI
  possibleSentence_str = json.loads(possibleSentences_list[i])
  possible_sentences = [item["value"] for item in possibleSentence_str][ : -1] # Remove 'Other'

  ## Checking if any sentence from the Intended Sentence list was not chosen
  ## There are two possible case scenarios:
  ## (1) The child skips a sentence
  ## (2) Identify SOME OF the cases with the second sentence segmentation error where a sentence is erroneously split into two in the Intended Sentence List
  if len(possible_sentences) > len(selected_sentences_sorted):
    case_issue = issues_list[i]
    has_segmentation_error = 'no'
    if type(case_issue) is not float:
      if 'segment' in case_issue or 'Segment' in case_issue or 'split' in case_issue or 'Split' in case_issue:
        has_segmentation_error = 'yes'

    if has_segmentation_error == 'yes':
      print(id_list[i])
      for z in range(len(possible_sentences)):
        try:
          if possible_sentences[z] == selected_sentences_sorted[z]:
            print('SAME', possible_sentences[z], selected_sentences_sorted[z])
          else:
            print('Different', possible_sentences[z], selected_sentences_sorted[z])
        except:
          print(possible_sentences[z])


      print('\n')

188964238
SAME Once upon a time, in a bustling snack bar, there was a bottle of soda named Joey. Once upon a time, in a bustling snack bar, there was a bottle of soda named Joey.
SAME Joey was excited because today was the day of the annual snack meeting. Joey was excited because today was the day of the annual snack meeting.
SAME All the snacks and drinks gathered to discuss the joy they bring to kids. All the snacks and drinks gathered to discuss the joy they bring to kids.
SAME Joey the soda bottle bounced with excitement, ready to join his friends. Joey the soda bottle bounced with excitement, ready to join his friends.
SAME As he rolled to the meeting, he spotted his best friend, a napkin named Roy. As he rolled to the meeting, he spotted his best friend, a napkin named Roy.
SAME Roy was always there to keep things tidy. Roy was always there to keep things tidy.
SAME "Oi, Joey!" "Oi, Joey!"
Different Roy called out, waving a corner of himself. "Hurry, or we'll be late for the meet

In [ ]:
for i in range(len(data)):

  ## Annotated time stamps
  SentenceLabel_str = ''
  try:
    SentenceLabel_str = json.loads(SentenceLabel_list[i])
  except:
    SentenceLabel_str = ast.literal_eval(SentenceLabel_list[i])

  ## Sort timestamps by start time
#  timestamps_sorted = sorted(SentenceLabel_str, key=lambda x: float(x['start']))

  ## Annotated Intended Sentences
  selected_sentences = ''
  try:
    selected_sentences = json.loads(SentenceSelect_list[i])
  except:
    try:
      selected_sentences = ast.literal_eval(SentenceSelect_list[i])
    except:
      ## When an audio has just one utterance, its intended sentence is returned as a single string, not a list with one string
      selected_sentences = [SentenceSelect_list[i]]


  # Sorting timestamps and annotated intended sentences together (ignore "Other")
  combined = [
    (ts, s) for ts, s in zip(SentenceLabel_str, selected_sentences) # This works even when lengths of timestamps and annotated intended sentences are different; therefore this step is after the Check above
    if s != "Other"
  ]

  # Sort by start time
  combined_sorted = sorted(combined, key=lambda x: float(x[0]["start"]))
  timestamps_sorted = [ts for ts, _ in combined_sorted]
  selected_sentences_sorted = [s for _, s in combined_sorted]

  ## Original old standard sentences
  goldStandardText_str = goldStandardText_list[i]
  gold_sentences = [line.split('. ', 1)[1] for line in goldStandardText_str.strip().split('\n')]

  ## Actual list of sentence segmentation used in the UI
  possibleSentence_str = json.loads(possibleSentences_list[i])
  possible_sentences = [item["value"] for item in possibleSentence_str][ : -1] # Remove 'Other'

  ## Checking if there is segmentation overlap between two adjacent utterances
  if len(SentenceLabel_str) == len(selected_sentences_sorted):
    for z in range(1, len(timestamps_sorted)): # for the current segmentation, compares it to the previous one
      prev_end = float(timestamps_sorted[z - 1]['end'])
      curr_start = float(timestamps_sorted[z]['start'])
      if curr_start < prev_end:
      #    print(f"{id_list[i]} ✗ Overlap between segment {z} and {z+1}: {curr_start:.2f} < {prev_end:.2f}")
        print(f"{id_list[i]} ✗ Overlap between Sentence {z} and {z + 1}") # z refers to the actual order of the sentence in the list of intended sentences, therefore it might not correspond to the segmend ID to the right side of the UI, if the annotators forgot to label a segment then later on added it
        print(annotator_list[i])
        try:
          print(timestamps_sorted[z-1], selected_sentences_sorted[z-1])
          print(timestamps_sorted[z], selected_sentences_sorted[z])
        except:
          print(id_list[i], z)
          print('ERROR')

          print(timestamps_sorted[z], selected_sentences_sorted[z])
          break
        print('\n')
